# Compare results for different dialects

In [ ]:
from pathlib import Path
from notebook_utils import get_formatted_score_df, make_heatmap
from asr_eval import utils
import pandas as pd


pd.options.mode.copy_on_write = True

In [ ]:
mean_score_file = Path("../data/output/2024/mean_scores_dialect.csv")

if not mean_score_file.exists():
    df = get_formatted_score_df(Path("../data/output/2024"))
    mean_score_df = utils.calculate_mean_scores(df, "dialect")

    mean_score_df.drop_duplicates(subset=["modell", "språk", "dialect"], inplace=True)
    mean_score_df.to_csv(mean_score_file, index=False)
else:
    mean_score_df = pd.read_csv(mean_score_file)

In [ ]:
imagedir = Path("images")
imagedir.mkdir(exist_ok=True, parents=True)

## Bokmål

In [ ]:
make_heatmap(
    mean_score_df,
    feature="dialect",
    metric="WER",
    language="nob",
    figsize=(12, 6),
    save_to_dir=imagedir,
)

## Nynorsk

In [ ]:
make_heatmap(
    mean_score_df, feature="dialect", metric="WER", language="nno", figsize=(12, 6)
)